In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from itertools import groupby, combinations
from operator import itemgetter
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
from tqdm.notebook import tqdm
sns.set_theme(style="darkgrid", rc={"grid.color": "0.6", "grid.alpha": "0.4"})
sns.set(font_scale=1.25)
plt.figure()
plt.show()

<Figure size 432x288 with 0 Axes>

In [2]:
# n = narrow, b = broad, t/f = true / false (predicted output does/doesn't match ground truth)

filetypes = ['nfbf-bicamnfbf', 'ntbf-bicamnfbf', 
                'nfbt-bicamnfbf', 'ntbt-bicamnfbf',
                'nfbf-bicamntbf', 'ntbf-bicamntbf', 
                'nfbt-bicamntbf', 'ntbt-bicamntbf',
                'nfbf-bicamnfbt', 'ntbf-bicamnfbt', 
                'nfbt-bicamnfbt', 'ntbt-bicamnfbt',
                'nfbf-bicamntbt', 'ntbf-bicamntbt', 
                'nfbt-bicamntbt', 'ntbt-bicamntbt',]

In [3]:
def get_index():
    for i in range(len(filetypes)):
        yield i

In [4]:
def get_feat(filetype):
    narrow_feat = np.load(f'./grad_cam_{filetype}/narrow_feat.npy').reshape(-1, 512)
    broad_feat = np.load(f'./grad_cam_{filetype}/broad_feat.npy').reshape(-1, 512)
    bicameral_feat = np.load(f'./grad_cam_{filetype}/bicameral_feat.npy').reshape(-1, 1024)
    return narrow_feat, broad_feat, bicameral_feat

In [5]:
def get_grouped_images(filetype):
    with open(f'./distribution/distribution_images_{filetype}.txt', 'r') as f:
        images = f.read().splitlines()
    images = [(ind, image.split('/')[-2]) for ind, image in enumerate(images)]
    grouped_images = [list(group) for key, group in groupby(images, itemgetter(1))]
    return grouped_images

In [6]:
def get_df(filetype):
    narrow_feat, broad_feat, bicameral_feat = get_feat(filetype)
    grouped_images = get_grouped_images(filetype)
    narrow_distances = []
    broad_distances = []
    bicameral_distances = []
    ensemble_distances = []
    labels = []

    df = pd.DataFrame(columns=['label', 'img_ind_i', 'img_ind_j', 'bicam', 'bicam_wo_sp', 'broad', 'narrow'])
    df_label, df_ind_i, df_ind_j, df_bicam, df_bicam_wo_sp, df_broad, df_narrow= [], [], [], [], [], [], []

    for group in grouped_images:
        if len(group) <= 1:
            narrow_distances.append([])
            broad_distances.append([])
            bicameral_distances.append([])
            ensemble_distances.append([])
            continue
        narrow_matrix = np.zeros((len(group), len(group)))
        broad_matrix = np.zeros((len(group), len(group)))
        bicameral_matrix = np.zeros((len(group), len(group)))
        ensemble_matrix = np.zeros((len(group), len(group)))
        labels.append(int(group[0][1]))
        for pair in list(combinations(enumerate(group), 2)):
            ind_i, ind_j = pair[0][0], pair[1][0]
            feat_ind_i, feat_ind_j = pair[0][1][0], pair[1][1][0]
            narrow_matrix[ind_i][ind_j] = sp.spatial.distance.cosine(
                                                    narrow_feat[feat_ind_i] , 
                                                    narrow_feat[feat_ind_j])
            broad_matrix[ind_i][ind_j] = sp.spatial.distance.cosine(
                                                    broad_feat[feat_ind_i], 
                                                    broad_feat[feat_ind_j])
            bicameral_matrix[ind_i][ind_j] = sp.spatial.distance.cosine(
                                                    bicameral_feat[feat_ind_i], 
                                                    bicameral_feat[feat_ind_j])
            ensemble_matrix[ind_i][ind_j] = sp.spatial.distance.cosine(
                                            np.hstack([narrow_feat[feat_ind_i], broad_feat[feat_ind_i]]), 
                                            np.hstack([narrow_feat[feat_ind_j], broad_feat[feat_ind_j]]))
            
            df_label.append(int(group[0][1][0]))
            df_ind_i.append(ind_i)
            df_ind_j.append(ind_j)
            df_bicam.append(bicameral_matrix[ind_i][ind_j])
            df_bicam_wo_sp.append(ensemble_matrix[ind_i][ind_j])
            df_broad.append(broad_matrix[ind_i][ind_j])
            df_narrow.append(narrow_matrix[ind_i][ind_j])

        narrow_distances.append(narrow_matrix)
        broad_distances.append(broad_matrix)
        bicameral_distances.append(bicameral_matrix)
        ensemble_distances.append(ensemble_matrix)

    df['label'] = df_label
    df['img_ind_i'] = df_ind_i
    df['img_ind_j'] = df_ind_j
    df['bicam'] = df_bicam
    df['bicam_wo_sp'] = df_bicam_wo_sp
    df['broad'] = df_broad
    df['narrow'] = df_narrow
    return df

### Save the dataframe

In [7]:
def save_df(df):
    df.to_csv(f'./grad_cam_{filetype}/cosine_analysis.csv')

In [8]:
def relplot(df, filetype):
    no_bins = 10
    
    plt.figure()
    no_bins_bicam = no_bins
    bicameral = df.copy()
    bicameral.drop(['label', 'img_ind_i', 'img_ind_j', 'bicam_wo_sp'], axis=1, inplace=True)
    bicam_labels = pd.cut(bicameral['bicam'], bins=no_bins, duplicates='drop').unique()
    if len(bicam_labels) < no_bins_bicam:
        no_bins_bicam = len(bicam_labels)
    bicam_labels = sorted([(bicam_labels[i].left + bicam_labels[i].right) / 2 for i in range(len(bicam_labels))])
    bicam_labels = [f"{i:.3f}" for i in bicam_labels]
    bicameral['size'] = pd.cut(-1*bicameral["bicam"], bins=no_bins_bicam, labels=bicam_labels[::-1])
    bicameral['feat'] = pd.cut(bicameral["bicam"], bins=no_bins_bicam, labels=bicam_labels)
    sns_relplot_bicam = sns.relplot(x="narrow", y="broad", hue="feat", data=bicameral, size="size", palette="RdYlGn", legend="brief", height=5, aspect=1)
    # sns_relplot_bicam.savefig(f"./grad_cam_{filetype}/{filetype}_bicam_feat_cosine_analysis_relplot.png", transparent=True)
    sns_relplot_bicam.savefig(f"./grad_cam_{filetype}/bicam_feat_cosine_analysis_relplot.png", transparent=False)
    plt.close()
    plt.clf()
    
    plt.figure()
    no_bins_bicam_wo_sp = no_bins
    bicameral_wo_sp = df.copy()
    bicameral_wo_sp.drop(['label', 'img_ind_i', 'img_ind_j', 'bicam'], axis=1, inplace=True)
    bicam_labels = pd.cut(bicameral_wo_sp['bicam_wo_sp'], bins=no_bins, duplicates='drop').unique()
    if len(bicam_labels) < no_bins_bicam_wo_sp:
        no_bins_bicam_wo_sp = len(bicam_labels)
    bicam_labels = sorted([(bicam_labels[i].left + bicam_labels[i].right) / 2 for i in range(len(bicam_labels))])
    bicam_labels = [f"{i:.3f}" for i in bicam_labels]
    bicameral_wo_sp['size'] = pd.cut(-1*bicameral_wo_sp["bicam_wo_sp"], bins=no_bins_bicam_wo_sp, labels=bicam_labels[::-1])
    bicameral_wo_sp['feat'] = pd.cut(bicameral_wo_sp["bicam_wo_sp"], bins=no_bins_bicam_wo_sp, labels=bicam_labels)
    sns_relplot_bicam_wo_sp = sns.relplot(x="narrow", y="broad", hue="feat", data=bicameral_wo_sp, size="size", palette="RdYlGn", legend="brief", height=5, aspect=1)
    # sns_relplot_bicam_wo_sp.savefig(f"./grad_cam_{filetype}/{filetype}_bicam_wo_sp_feat_cosine_analysis_relplot.png", transparent=True)
    sns_relplot_bicam_wo_sp.savefig(f"./grad_cam_{filetype}/bicam_wo_sp_feat_cosine_analysis_relplot.png", transparent=False)
    plt.close()
    plt.clf()

In [19]:
def kdeplot(df, filetype):
    no_bins = 10

    X_plot = np.linspace(0, 0.5, 50)
    Y_plot = X_plot
    
    # try:
    plt.figure()
    no_bins_bicam = no_bins
    bicameral = df.copy()
    bicameral.drop(['label', 'img_ind_i', 'img_ind_j', 'bicam_wo_sp'], axis=1, inplace=True)
    bicam_labels = pd.cut(bicameral['bicam'], bins=no_bins).unique()
    if len(bicam_labels) < no_bins_bicam:
        no_bins_bicam = len(bicam_labels)
    bicam_labels = sorted([(bicam_labels[i].left + bicam_labels[i].right) / 2 for i in range(len(bicam_labels))])
    bicam_labels = [f"{i:.3f}" for i in bicam_labels]
    bicameral['size'] = pd.cut(-1*bicameral["bicam"], bins=no_bins_bicam, labels=bicam_labels[::-1])
    bicameral['bicameral features cosine value'] = pd.cut(bicameral["bicam"], bins=no_bins_bicam, labels=bicam_labels)
    sns_kdeplot_bicam = sns.JointGrid(data=bicameral, x="narrow", y="broad", hue="bicameral features cosine value", palette="RdYlGn", height=5)
    sns_kdeplot_bicam.plot(sns.scatterplot, sns.histplot)
    sns_kdeplot_bicam.fig.subplots_adjust(top=0.9)
    # sns_kdeplot_bicam.fig.suptitle('Cosine Values')
    sns_kdeplot_bicam.set_axis_labels('Left Hemisphere (Specific)', 'Right Hemisphere (General)')
    if sns_kdeplot_bicam.ax_joint.get_legend() is not None:
        sns_kdeplot_bicam.ax_joint.legend_.set_bbox_to_anchor(bbox=[1.65, 1.1])
        sns_kdeplot_bicam.ax_joint.legend_.set_title(title='Bilateral')
        plt.setp(sns_kdeplot_bicam.ax_joint.legend_.get_title(), fontsize='15')
        plt.setp(sns_kdeplot_bicam.ax_joint.legend_.get_texts(), fontsize='15')
    ax = plt.gca()
    sns_kdeplot_bicam.ax_joint.plot(X_plot, Y_plot, color='b', linestyle='--', linewidth=1, alpha=0.4)
    sns_kdeplot_bicam.savefig(f"./grad_cam_{filetype}/{filetype}_bicam_feat_cosine_analysis_kdeplot.png", transparent=False, dpi=1500)
    plt.close()
    plt.clf()
    
    plt.figure()
    no_bins_bicam_wo_sp = no_bins
    bicameral_wo_sp = df.copy()
    bicameral_wo_sp.drop(['label', 'img_ind_i', 'img_ind_j', 'bicam'], axis=1, inplace=True)
    bicam_labels = pd.cut(bicameral_wo_sp['bicam_wo_sp'], bins=no_bins).unique()
    if len(bicam_labels) < no_bins_bicam_wo_sp:
        no_bins_bicam_wo_sp = len(bicam_labels)
    
    bicam_labels = sorted([(bicam_labels[i].left + bicam_labels[i].right) / 2 for i in range(len(bicam_labels))])
    bicam_labels = [f"{i:.3f}" for i in bicam_labels]
    bicameral['size'] = pd.cut(-1*bicameral["bicam"], bins=no_bins_bicam_wo_sp, labels=bicam_labels[::-1])
    bicameral_wo_sp['features cosine value'] = pd.cut(bicameral_wo_sp["bicam_wo_sp"], bins=no_bins_bicam_wo_sp, labels=bicam_labels)
    sns_kdeplot_bicam_wo_sp = sns.JointGrid(data=bicameral_wo_sp, x="narrow", y="broad", hue="features cosine value", palette="RdYlGn", height=5)
    sns_kdeplot_bicam_wo_sp.plot(sns.scatterplot, sns.histplot)
    sns_kdeplot_bicam_wo_sp.fig.subplots_adjust(top=0.9)
    # sns_kdeplot_bicam_wo_sp.fig.suptitle('Cosine Values')
    sns_kdeplot_bicam_wo_sp.set_axis_labels('Left Hemisphere (Specific)', 'Right Hemisphere (General)')
    
    if sns_kdeplot_bicam_wo_sp.ax_joint.get_legend() is not None:
        sns_kdeplot_bicam_wo_sp.ax_joint.legend_.set_bbox_to_anchor(bbox=[1.75, 1.1])
        sns_kdeplot_bicam_wo_sp.ax_joint.legend_.set_title(title='Concatenated')
        plt.setp(sns_kdeplot_bicam_wo_sp.ax_joint.legend_.get_title(), fontsize='15')
        plt.setp(sns_kdeplot_bicam_wo_sp.ax_joint.legend_.get_texts(), fontsize='15')
    ax = plt.gca()
    sns_kdeplot_bicam_wo_sp.ax_joint.plot(X_plot, Y_plot, color='b', linestyle='--', linewidth=1, alpha=0.4)
    sns_kdeplot_bicam_wo_sp.savefig(f"./grad_cam_{filetype}/{filetype}_bicam_wo_sp_feat_cosine_analysis_kdeplot.png", transparent=False, dpi=1500)
    plt.close()
    plt.clf()

In [20]:
iter_index = get_index()
for index in iter_index:
    # if index != 12:
    #     continue
    filetype = filetypes[index]
    print(f"{index+1} {filetype}", end="\n")
    df = get_df(filetype)
    # save_df(df)
    kdeplot(df, filetype)
    # relplot(df, filetype)
    

1 nfbf-bicamnfbf
2 ntbf-bicamnfbf
3 nfbt-bicamnfbf
4 ntbt-bicamnfbf
5 nfbf-bicamntbf
6 ntbf-bicamntbf
7 nfbt-bicamntbf
8 ntbt-bicamntbf
9 nfbf-bicamnfbt
10 ntbf-bicamnfbt


/home/chandramouli/miniconda3/envs/pytorch_p38/lib/python3.8/site-packages/seaborn/axisgrid.py:1670: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  f = plt.figure(figsize=(height, height))


11 nfbt-bicamnfbt


/tmp/ipykernel_8742/379515018.py:8: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure()


12 ntbt-bicamnfbt
13 nfbf-bicamntbt
14 ntbf-bicamntbt
15 nfbt-bicamntbt
16 ntbt-bicamntbt


...

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

: 